In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             classification_report, confusion_matrix)
import transformers
from transformers import (AutoTokenizer , AutoModelForSequenceClassification, TrainingArguments, 
                          Trainer, pipeline, DataCollatorWithPadding, AutoModelForSeq2SeqLM, 
                          EarlyStoppingCallback, IntervalStrategy)
from datasets import load_dataset, Dataset, load_metric
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
from torch.utils.tensorboard import SummaryWriter
import random
import wandb
import collections
CUDA_LAUNCH_BLOCKING=1
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import StratifiedKFold
import collections
import mlflow
from mlflow.tracking import MlflowClient

2023-05-30 10:07:15.517645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 10:07:16.188139: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-30 10:07:16.188176: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-30 10:07:16.293789: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-30 10:07:17.693705: W tensorflow/stream_executor/pla

In [5]:
print(torch.cuda.is_available())

True


In [6]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 11% | 13% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 24% | 14% |


In [7]:
max_length = 256
def load_data(train_file_path, valid_file_path):
    if train_file_path and valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path, 'validation': valid_file_path})
    elif not valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path})
    return dataset
model_checkpoint = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
def tokenize_data(example):
    return tokenizer(preprocess(example['text']),  padding = 'max_length', truncation=True, max_length=max_length)
#training and validation - 60/40
#dataset = load_data(['RT_train.csv'],  ['RT_test.csv'])
#train_tokenized = dataset['train'].map(tokenize_data, batched = True) 
#valid_tokenized = dataset['validation'].map(tokenize_data, batched = True)

# training with whole data for deployment
#dataset = load_data(['RT_train.csv', 'RT_test.csv'], None)
#train_tokenized = dataset['train'].map(tokenize_data, batched = True)

#training with whole data + 50% coded 2021 data, validating on 50% coded 2021
#dataset = load_data(['RT_train.csv','RT_test.csv', 'RT_train_2021_j.csv'], ['RT_test_2021_j.csv'])
#dataset = load_data(['RT_j.csv', 'RT_train_2021_j.csv'], ['RT_test_2021_j.csv'])
#train_tokenized = dataset['train'].map(tokenize_data, batched = True)
#valid_tokenized = dataset['validation'].map(tokenize_data, batched = True)

#training with whole data + 50% coded 2021 data, validating on 50% coded 2021, removing No Abstract Available (second)
#dataset = load_data(['RT_train.csv','RT_test.csv', 'RT_train_2021_j.csv'], ['RT_test_2021_j.csv'])
#dataset = load_data(['RT_j_second.csv', 'RT_train_2021_j_second.csv'], ['RT_test_2021_j_second.csv'])
#train_tokenized = dataset['train'].map(tokenize_data, batched = True)
#valid_tokenized = dataset['validation'].map(tokenize_data, batched = True)

#training with whole data + 50% coded 2021 data, validating on 50% coded 2021, separate abstract and title
#dataset = load_data(['RT_s.csv', 'RT_train_2021_s.csv', 'RT_test_2021_s.csv'], None)


In [8]:
#prevention
max_length = 512
def load_data(train_file_path, valid_file_path):
    if train_file_path and valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path, 'validation': valid_file_path})
    elif not valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path})
    return dataset
model_checkpoint = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
def tokenize_data(example):
    return tokenizer(preprocess(example['text']),  padding = 'max_length', truncation=True, max_length=max_length)
dataset = load_data(['trainPrevention.csv'],  ['validPrevention.csv'])
train_tokenized = dataset['train'].map(tokenize_data, batched = True) 
valid_tokenized = dataset['validation'].map(tokenize_data, batched = True)

Using custom data configuration default-ebf4f7f31e28ef9f
Found cached dataset csv (/home/elahehaa/.cache/huggingface/datasets/csv/default-ebf4f7f31e28ef9f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

NameError: name 'preprocess' is not defined

In [8]:
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=4,
        #id2label={index: label for index, label in enumerate(labels.names)},
        #label2id={label: index for index, label in enumerate(labels.names)}
    )
    return model
model = model_init()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [ ]:
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'values': [1,2,4]
        },
    'batch_size': {
        'values': [4, 8, 16]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 2e-7,
        'max': 2e-4
    },
    'weight_decay': {
        'values': [0.0, 0.01, 0.005, 0.001]
    },
}


sweep_config['parameters'] = parameters_dict


In [ ]:
sweep_id = wandb.sweep(sweep_config, project='ft_all+PK_PubMedBERT3')

In [9]:
#tokenize/process title and abstract separately
def customized_data_collator(examples):
    titles = [example['Title'] for example in examples]
    abstracts = [example['Abstract'] for example in examples]
    
    tokenized_title = tokenizer(titles , padding = 'max_length', truncation=True, max_length=30)
    tokenized_abstract = tokenizer(abstracts, padding = 'max_length', truncation=True, max_length=256)
    
    #inputs = {
        #'input_ids' : torch.cat([tokenized_title['input_ids'] , tokenized_abstract['input_ids']], dim =1),
        #'attention_mask': torch.cat([tokenized_title['attention_mask'] , tokenized_abstract['attention_mask']], dim =1),
        #'token_type_ids' : torch.cat([tokenized_title['token_type_ids'] , tokenized_abstract['token_type_ids']], dim =1)
    #}
    
    inputs = {
        'input_ids' : [a+b for a,b in zip(tokenized_title['input_ids'],tokenized_abstract['input_ids'])],
        'attention_mask': [a+b for a,b in zip(tokenized_title['attention_mask'],tokenized_abstract['attention_mask'])],
        'token_type_ids' : [a+b for a,b in zip(tokenized_title['token_type_ids'],tokenized_abstract['token_type_ids'])]
    }
    
    label = [example['labels'] for example in examples]
    batch = {
        'input_ids': torch.tensor(inputs['input_ids']),
        'attention_mask': torch.tensor(inputs['attention_mask']),
        'token_type_ids': torch.tensor(inputs['token_type_ids']),
        'labels': torch.tensor(label)
    }
    
    return batch
#print(customized_data_collator(dataset['train']))

In [ ]:
#a test to find if tokenize and then merge title and abstract values will be different from first merge and then tokenize
tokenized_t = tokenizer(dataset['train']['Title'][15] , padding = 'max_length', truncation=True, max_length=max_length)
tokenized_a = tokenizer(dataset['train']['Abstract'][15] , padding = 'max_length', truncation=True, max_length=max_length)
tokenized_ta = tokenizer(dataset['train']['Title'][15]+dataset['train']['Abstract'][15] , padding = 'max_length', truncation=True, max_length=max_length)
print([a+b for a,b in zip(tokenized_t['input_ids'],tokenized_a['input_ids'])])
print(tokenized_ta['input_ids'])


In [10]:
# define function to compute metrics
def compute_metrics_fn(eval_preds):
    
    res = dict()

    accuracy_metric = load_metric('accuracy')
    precision_metric = load_metric('precision')
    recall_metric = load_metric('recall')
    f1_metric = load_metric('f1')
    
    logits = eval_preds.predictions
    labels = eval_preds.label_ids
    preds = np.argmax(logits, axis=-1)  
    
    report = classification_report(labels, preds)
    cm = confusion_matrix(labels, preds)
    print(report)
    print('confusion matrix: ', cm)
    
    res.update(accuracy_metric.compute(predictions=preds, references=labels))
    res.update(precision_metric.compute(predictions=preds, references=labels, average='macro'))
    res.update(recall_metric.compute(predictions=preds, references=labels, average='macro'))
    res.update(f1_metric.compute(predictions=preds, references=labels, average='macro'))
    
    return res 


In [9]:
#experiments using 60/40 - hyperparam tuning
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config


        training_args = TrainingArguments(
            output_dir='ft_all+PK_PubMedBERT3',
            report_to='wandb', 
            num_train_epochs=config.epochs,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=16,
            save_strategy='epoch',
            evaluation_strategy='epoch',
            logging_strategy='epoch',
            load_best_model_at_end=True,
            remove_unused_columns=False,
            fp16=True,
            save_total_limit = 1,
            run_name= 'ft_all+PK_PubMedBERT3',
        )
        

        trainer = Trainer(
            model_init=model_init,
            args=training_args,
            train_dataset=train_tokenized,
            eval_dataset=valid_tokenized,
            compute_metrics=compute_metrics_fn
        )


        trainer.train()


In [ ]:
wandb.agent(sweep_id, train, count=15)

In [8]:
#candidate model more epochs
def train(learning_rate, weight_decay, run_name, epochs, batch_size):
    training_args = TrainingArguments(
        output_dir= run_name,
        report_to='wandb', 
        num_train_epochs= epochs,
        learning_rate= learning_rate,
        weight_decay= weight_decay,
        per_device_train_batch_size= batch_size,
        per_device_eval_batch_size=16,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,
        remove_unused_columns=False,
        #fp16=True,
        save_total_limit = 1,
        run_name= run_name,
    )


    trainer = Trainer(
        model_init= model_init,
        args= training_args,
        train_dataset= train_tokenized,
        eval_dataset= valid_tokenized,
        compute_metrics= compute_metrics_fn
    )


    trainer.train()
#selected1
#train(4.495e-5, 0, "ft_all_PubMedBERT_selected1", 10, 16)
#selected2
#train(2.896e-5, 0.01, "ft_all_PubMedBERT_selected2", 10, 8)
#selected3 - candidate
#train(1.098e-4, 0.01, "ft_all_PubMedBERT_selected32", 10, 8)
#PubMedBERT3selected
#train(4.3e-5, 0.01, "ft_all+PK_PubMedBERT3_selected2", 4, 16)

#prevention
train(1.098e-4, 0.01, "ft_prevention_PubMedBERT", 1, 8)

NameError: name 'train_tokenized' is not defined

In [40]:
#train final model with all data - test with early stopping , return to normal if did not work, add class weights
torch.manual_seed(42)
class CustomTrainer(Trainer):
    def calculate_class_weights(training_set):
        print("hi")
        labels = [set(training_set)]
        class_distribution = [0]*len(labels)
        for i in labels:
            class_distribution[i] = training_set.count(i)
        weights = []
        class_distribution = np.array(class_distribution)
        num_classes = len(labels)
        weight = np.sum(class_distribution)/(num_classes * class_distribution)
        return weight
    def compute_custom_loss(model, inputs, return_outputs=False):
        target = inputs.get('labels')
        weights = calculate_class_weights(target)
        print(weights)
        print("hi2")
        ce_loss = nn.CrossEntropyLoss(weight=torch.tensor(weights))
        outputs = model(*inputs)
        logits = outputs.get('logits')
        probs = nn.functional.softmax(logits, dim = -1)
        
        loss = ce_loss(probs.view(-1, model.config.num_labels), target.view(-1))
        return (loss, outputs) if return_outputs else loss
def train(train_set, validation_set, learning_rate, weight_decay, run_name, epochs, batch_size):
    mlflow.start_run()

    # Log training parameters
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", epochs)

    training_args = TrainingArguments(
        output_dir= run_name,
        num_train_epochs= epochs,
        learning_rate= learning_rate,
        weight_decay= weight_decay,
        per_device_train_batch_size= batch_size,
        save_strategy= 'epoch' ,#IntervalStrategy.STEPS,
        logging_strategy= 'epoch', #IntervalStrategy.STEPS,
        remove_unused_columns=False,
        save_total_limit = 1,
        run_name= run_name,
        fp16 = True,
        load_best_model_at_end=True,
        evaluation_strategy = 'epoch', #IntervalStrategy.STEPS,
        eval_steps = 1,
        metric_for_best_model = 'eval_f1',
        seed = 42
    )


    trainer = CustomTrainer(
        model_init= model_init,
        args= training_args,
        #train_tokenized for joined title and abstract, dataset['train'] for seperate title and abstract
        train_dataset= train_set,
        #use to create seperate title and abstract
        data_collator = customized_data_collator,
        compute_metrics= compute_metrics_fn,
        #valid_tokenized for joined title and abstract, dataset['validation'] for seperate title and abstract
        eval_dataset = validation_set,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
        
    )


    trainer.train()
    eval_result = trainer.evaluate()
    mlflow.end_run()

    return eval_result


#final model training with all data
#train(1.098e-4, 0.01, "final_PubMedBERT_longer_longerepoch", 5, 8)
#final model training with all data + 50% coded 2021 data, joined abstract and title
#train(1.098e-4, 0.01, "final_PubMedBERT_2021",4, 8)
#final model training with all data + 50% coded 2021 data , separete title and abstract
#train(train_set, test_set, 1.098e-4, 0.01, "final_PubMedBERT_2021_separateInput", 5, 8)

#final model training with all data + 50% coded 2021 data , separate title and abstract
#final = train(train_set, test_set, 1.098e-4, 0.01, "final_PubMedBERT_2021_separateInput_v3", 6, 8)
#final = train(train_set, test_set, 1.098e-4, 0.01, "final_PubMedBERT_2021_separateInput_test2", 20, 8)


In [16]:
#stratified sampling to make sure enough sample from each class exists in the classification
mlflow.end_run()
# Set the experiment path
experiment_path = "PubClassifier"

# Set the experiment
mlflow.set_experiment(experiment_path)
#stratified k_fold cross validation for imbalanced dataset
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#heldout_set
k_fold , test = next(skf.split(dataset['train'], dataset['train']['labels']))
#kfold training and validation
eval_result = []
i = 0
for train_idx , valid_idx in skf.split(dataset['train'][k_fold]['Title'], dataset['train'][k_fold]['labels']):
    train_set = [dataset['train'][int(i)] for i in train_idx]
    validation_set = [dataset['train'][int(i)] for i in valid_idx]
    eval_result.append(train(train_set, validation_set, 3.098e-5, 0.001, "PubMedBERT_2021_lr35"+str(i), 10, 16))
    i += 1
    

#1.098e-4, 0.01

In [10]:
#class weights to imptove IRGT class performance
class CustomTrainer(Trainer):
    def calculate_class_weights(training_set):
        print("hi")
        labels = [set(training_set)]
        class_distribution = [0]*len(labels)
        for i in labels:
            class_distribution[i] = training_set.count(i)
        weights = []
        class_distribution = np.array(class_distribution)
        num_classes = len(labels)
        weight = np.sum(class_distribution)/(num_classes * class_distribution)
        return weight
    def compute_custom_loss(model, inputs, return_outputs=False):
        target = inputs.get('labels')
        weights = calculate_class_weights(target)
        print(weights)
        print("hi2")
        ce_loss = nn.CrossEntropyLoss(weight=torch.tensor(weights))
        outputs = model(*inputs)
        logits = outputs.get('logits')
        probs = nn.functional.softmax(logits, dim = -1)
        
        loss = ce_loss(probs.view(-1, model.config.num_labels), target.view(-1))
        return (loss, outputs) if return_outputs else loss
    
def train(train_set, validation_set, learning_rate, weight_decay, run_name, epochs, batch_size):
    training_args = TrainingArguments(
        output_dir= run_name,
        num_train_epochs= epochs,
        learning_rate= learning_rate,
        weight_decay= weight_decay,
        per_device_train_batch_size= batch_size,
        save_strategy='epoch',
        logging_strategy='epoch',
        remove_unused_columns=False,
        save_total_limit = 1,
        run_name= run_name,
        fp16 = True
    )


    trainer = CustomTrainer(
        model_init= model_init,
        args= training_args,
        #train_tokenized for joined title and abstract, dataset['train'] for seperate title and abstract
        train_dataset= train_set,
        #use to create seperate title and abstract
        data_collator = customized_data_collator,
        compute_metrics= compute_metrics_fn,
        #valid_tokenized for joined title and abstract, dataset['validation'] for seperate title and abstract
        eval_dataset = validation_set,
        
    )


    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result


#final model training with all data
#train(1.098e-4, 0.01, "final_PubMedBERT_longer_longerepoch", 5, 8)
#final model training with all data + 50% coded 2021 data, joined abstract and title
#train(1.098e-4, 0.01, "final_PubMedBERT_2021",4, 8)
#final model training with all data + 50% coded 2021 data , separete title and abstract
#train(train_set, test_set, 1.098e-4, 0.01, "final_PubMedBERT_2021_separateInput", 5, 8)

#final model training with all data + 50% coded 2021 data , separate title and abstract
final = train(train_set, test_set, 1.098e-4, 0.01, "final_PubMedBERT_2021_separateInput_test", 5, 8)


NameError: name 'train_set' is not defined

In [17]:
#stratified train and test set
test_set = [dataset['train'][int(i)] for i in test]
train_set = [dataset['train'][int(i)] for i in k_fold]

In [28]:
#evaluate candidate model on test set
train(train_set, test_set, 3.098e-5, 0.001, "candidate_PubMedBERT_2021_test2", 4, 16)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/resolve/main/config.json from cache at /home/elahehaa/.cache/huggingface/transformers/76e7b0967140f134278c3209cffe98f69eb013b9de505a434b3359c057aedaa3.2411d0fafcf181e9b95d9cb7972d93b27c57a2cb75819924f8fc7ec848b708f2
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,0,1,2,3
1,0.448500,0.257617,0.905738,0.686920,0.709270,0.696922,0.648148,0.209677,1.000000,0.500000
2,0.180900,0.283669,0.919399,0.833679,0.856207,0.844052,0.647059,0.369565,0.421053,0.500000
3,0.089600,0.311875,0.938525,0.883923,0.874187,0.878921,0.432432,0.552632,0.538462,0.500000
4,0.047200,0.313196,0.935792,0.889207,0.875452,0.881781,0.575000,0.400000,0.583333,0.500000


***** Running Evaluation *****
  Num examples = 732
  Batch size = 8


              precision    recall  f1-score   support

           0       0.96      0.92      0.94       439
           1       0.82      0.95      0.88       238
           2       0.00      0.00      0.00        20
           3       0.97      0.97      0.97        35

    accuracy                           0.91       732
   macro avg       0.69      0.71      0.70       732
weighted avg       0.89      0.91      0.89       732



/home/elahehaa/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elahehaa/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elahehaa/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elahehaa/anaconda3/

...

Saving model checkpoint to PubMedBERT_2021_test2/checkpoint-549
Configuration saved in PubMedBERT_2021_test2/checkpoint-549/config.json
Model weights saved in PubMedBERT_2021_test2/checkpoint-549/pytorch_model.bin
Deleting older checkpoint [PubMedBERT_2021_test2/checkpoint-366] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 732
  Batch size = 8


              precision    recall  f1-score   support

           0       0.96      0.95      0.95       439
           1       0.90      0.93      0.92       238
           2       0.72      0.65      0.68        20
           3       0.97      0.97      0.97        35

    accuracy                           0.94       732
   macro avg       0.89      0.88      0.88       732
weighted avg       0.94      0.94      0.94       732



Saving model checkpoint to PubMedBERT_2021_test2/checkpoint-732
Configuration saved in PubMedBERT_2021_test2/checkpoint-732/config.json
Model weights saved in PubMedBERT_2021_test2/checkpoint-732/pytorch_model.bin
Deleting older checkpoint [PubMedBERT_2021_test2/checkpoint-549] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from PubMedBERT_2021_test2/checkpoint-732 (score: 0.8817807275030399).
***** Running Evaluation *****
  Num examples = 732
  Batch size = 8


              precision    recall  f1-score   support

           0       0.96      0.95      0.95       439
           1       0.90      0.93      0.92       238
           2       0.72      0.65      0.68        20
           3       0.97      0.97      0.97        35

    accuracy                           0.94       732
   macro avg       0.89      0.88      0.88       732
weighted avg       0.94      0.94      0.94       732



{'eval_loss': 0.31319618225097656,
 'eval_accuracy': 0.9357923497267759,
 'eval_precision': 0.8892072120160331,
 'eval_recall': 0.8754524702819625,
 'eval_f1': 0.8817807275030399,
 'eval_0': 0.575,
 'eval_1': 0.4,
 'eval_2': 0.5833333333333334,
 'eval_3': 0.5,
 'eval_runtime': 14.6419,
 'eval_samples_per_second': 49.993,
 'eval_steps_per_second': 6.283,
 'epoch': 4.0}

In [9]:
#deploy
torch.manual_seed(42)
class CustomTrainer(Trainer):
    def calculate_class_weights(training_set):
        labels = [set(training_set)]
        class_distribution = [0]*len(labels)
        for i in labels:
            class_distribution[i] = training_set.count(i)
        weights = []
        class_distribution = np.array(class_distribution)
        num_classes = len(labels)
        weight = np.sum(class_distribution)/(num_classes * class_distribution)
        return weight
    def compute_custom_loss(model, inputs, return_outputs=False):
        target = inputs.get('labels')
        weights = calculate_class_weights(target)
        ce_loss = nn.CrossEntropyLoss(weight=torch.tensor(weights))
        outputs = model(*inputs)
        logits = outputs.get('logits')
        probs = nn.functional.softmax(logits, dim = -1)
        
        loss = ce_loss(probs.view(-1, model.config.num_labels), target.view(-1))
        return (loss, outputs) if return_outputs else loss
def train(train_set, learning_rate, weight_decay, run_name, epochs, batch_size):
    mlflow.start_run()

    # Log training parameters
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", epochs)

    training_args = TrainingArguments(
        output_dir= run_name,
        num_train_epochs= epochs,
        learning_rate= learning_rate,
        weight_decay= weight_decay,
        per_device_train_batch_size= batch_size,
        save_strategy= 'epoch' ,#IntervalStrategy.STEPS,
        logging_strategy= 'epoch', #IntervalStrategy.STEPS,
        remove_unused_columns=False,
        save_total_limit = 1,
        run_name= run_name,
        fp16 = True,
        #load_best_model_at_end=True,
        #evaluation_strategy = 'epoch', #IntervalStrategy.STEPS,
        metric_for_best_model = 'eval_f1',
        seed = 42
    )


    trainer = CustomTrainer(
        model_init= model_init,
        args= training_args,
        #train_tokenized for joined title and abstract, dataset['train'] for seperate title and abstract
        train_dataset= train_set,
        #use to create seperate title and abstract
        data_collator = customized_data_collator,
        compute_metrics= compute_metrics_fn,
        #valid_tokenized for joined title and abstract, dataset['validation'] for seperate title and abstract
        
    )


    trainer.train()
    mlflow.end_run()

mlflow.end_run()
# Set the experiment path
experiment_path = "PubClassifier"
# Set the experiment
mlflow.set_experiment(experiment_path)

#train model on all available data to classify unseen 2022 data
train(dataset['train'], 3.098e-5, 0.001, "candidate_PubMedBERT_2021_5/27/23", 4, 16)

loading configuration file https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/resolve/main/config.json from cache at /home/elahehaa/.cache/huggingface/transformers/76e7b0967140f134278c3209cffe98f69eb013b9de505a434b3359c057aedaa3.2411d0fafcf181e9b95d9cb7972d93b27c57a2cb75819924f8fc7ec848b708f2
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "nu

Step,Training Loss
229,0.387200
458,0.169700
687,0.092500
916,0.044700


Saving model checkpoint to candidate_PubMedBERT_2021_5/27/23/checkpoint-229
Configuration saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-229/config.json
Model weights saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-229/pytorch_model.bin
Saving model checkpoint to candidate_PubMedBERT_2021_5/27/23/checkpoint-458
Configuration saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-458/config.json
Model weights saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-458/pytorch_model.bin
Deleting older checkpoint [candidate_PubMedBERT_2021_5/27/23/checkpoint-229] due to args.save_total_limit
Saving model checkpoint to candidate_PubMedBERT_2021_5/27/23/checkpoint-687
Configuration saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-687/config.json
Model weights saved in candidate_PubMedBERT_2021_5/27/23/checkpoint-687/pytorch_model.bin
Deleting older checkpoint [candidate_PubMedBERT_2021_5/27/23/checkpoint-458] due to args.save_total_limit
Saving model checkpoint to candidate_P

In [ ]:
#experiments using 60/40
run_name = "ft_PubMedBert_all_lr46_13"
def model_init(num_labels, model_checkpoint):
    return  AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = num_labels)


     
def fine_tune( model_checkpoint, output_dir,  epochs,  num_labels, config = None):
    
    with wandb.init(config=config):
    # set sweep configuration
        config = wandb.config

        training_args = TrainingArguments(
            output_dir = output_dir,
            report_to='wandb',
            learning_rate = config.learning_rate,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            num_train_epochs= epochs,
            weight_decay = config.weight_decay,
            #logging_steps = 'epoch',
            load_best_model_at_end = True,
            evaluation_strategy = 'epoch',
            save_strategy= 'epoch',
            save_total_limit = 1,
            run_name= run_name
            )

        trainer = Trainer(
        model=model_init(num_labels = num_labels, model_checkpoint = num_checkpoint),
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=valid_tokenized,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics = compute_metrics
        #callbacks = [early_stopping],

         )
        a = trainer.train()
    #wandb.finish()
    return a

fine_tune( model_checkpoint = model_checkpoint, output_dir = run_name,  num_labels = 4, epochs = 30,)
#%tensorboard --logdir logs/fit/

In [ ]:
#fine-tuning for final classification with whole data
run_name = "ftw_PubMedBert_GRT_256_lr77_13_2"
def fine_tune(model_checkpoint, output_dir, lr, batch_size, epochs, weight_decay):
    seed = 123
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2)
    training_args = TrainingArguments(
        output_dir = output_dir,
        learning_rate = lr,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs= epochs,
        weight_decay = weight_decay,
        #report_to='wandb',
        #logging_steps = 'epoch',
        #load_best_model_at_end = True,
        #evaluation_strategy = 'epoch',
        save_strategy= 'epoch',
        save_total_limit = 1,
        #run_name= run_name,
        
        )

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    #eval_dataset=valid_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #callbacks = [early_stopping],
     )
    a = trainer.train()
    #wandb.finish()
    return a

fine_tune(model_checkpoint, "ftw_PubMedBert_GRT_256_lr77_13_2", 7e-7, 4, 12, 0.001)

In [ ]:
#experiments
ft_model = AutoModelForSequenceClassification.from_pretrained("ft_all_PubMedBERT_selected32/checkpoint-465")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
clf = pipeline("text-classification", model = ft_model, tokenizer = tokenizer)

In [12]:
#deployment
def load_data(unknown_file_path):
    dataset = load_dataset("csv", data_files = {'unknown': unknown_file_path})
    return dataset

#ft_model = AutoModelForSequenceClassification.from_pretrained("final_PubMedBERT/checkpoint-771")
#ft_model = AutoModelForSequenceClassification.from_pretrained("final_PubMedBERT_longer/checkpoint-771")
#ft_model = AutoModelForSequenceClassification.from_pretrained("final_PubMedBERT_2021/checkpoint-1785")
ft_model = AutoModelForSequenceClassification.from_pretrained("candidate_PubMedBERT_2021_5/27/23/checkpoint-916")

def tokenize_data(example):
    return tokenizer(example['text'],  padding = True, truncation=True, max_length=max_length)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#dataset = load_data(["2021pubs_cleaned.csv"])
#unknown_tokenized = dataset['unknown'].map(tokenize_data, batched = True)
dataset = load_data(["2022pubs_cleaned_sep.csv"])
unknown_tokenized = dataset['unknown'].map(tokenize_data, batched = True)
clf = pipeline("text-classification", model = ft_model, tokenizer = tokenizer)

Using custom data configuration default-a7fd4db854b56d28
Found cached dataset csv (/home/elahehaa/.cache/huggingface/datasets/csv/default-a7fd4db854b56d28/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

KeyError: 'text'

In [ ]:
#experiments
def res(example):
    return clf(example['text'],  truncation = True)
#predictions = dataset['test'].map(res, batched = True)
#print(predictions)
predictions = {}
for i in range(len(dataset['validation'])):
    predictions[i] = clf(dataset['validation']['text'][i], truncation = 'longest_first', max_length = max_length)[0]['label']
#for i in range(12, len(dataset['validation'])):
    #print(i)
    #predictions[i] = clf(dataset['validation']['text'][i], truncation = 'longest_first', max_length = 20)[0]['label']
print (predictions)

In [31]:
#prediction for models processing title and abstract separately
#ft_model = AutoModelForSequenceClassification.from_pretrained("PubMedBERT_2021_test/checkpoint-1098")
#ft_model = AutoModelForSequenceClassification.from_pretrained("PubMedBERT_2021_test/checkpoint-915")
#ft_model = AutoModelForSequenceClassification.from_pretrained("PubMedBERT_2021_test2/checkpoint-732")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#predict 2022
ft_model = AutoModelForSequenceClassification.from_pretrained("candidate_PubMedBERT_2021_5/27/23/checkpoint-916")


def predict(examples):
    results = collections.defaultdict(list)
    titles = [example['Title'] for example in examples]
    abstracts = [example['Abstract'] for example in examples]
    pmids = [example['PMID'] for example in examples]
    #labels = [example['labels'] for example in examples]
    tokenized_title = tokenizer(titles, padding='max_length', truncation=True, max_length=50)
    tokenized_abstract = tokenizer(abstracts, padding='max_length', truncation=True, max_length=256)
    
    inputs = {
        'input_ids': [a + b for a, b in zip(tokenized_title['input_ids'], tokenized_abstract['input_ids'])],
        'attention_mask': [a + b for a, b in zip(tokenized_title['attention_mask'], tokenized_abstract['attention_mask'])],
        'token_type_ids': [a + b for a, b in zip(tokenized_title['token_type_ids'], tokenized_abstract['token_type_ids'])]
    }
    
    # Create input tensors
    input_ids = torch.tensor(inputs['input_ids'])
    attention_mask = torch.tensor(inputs['attention_mask'])
    token_type_ids = torch.tensor(inputs['token_type_ids'])
    
    # Perform the prediction
    with torch.no_grad():
        outputs = ft_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
    predictions = outputs.logits.argmax(dim=1).tolist()
    #for pm, label, pred in zip(pmids, labels, predictions):
        #results[pm].append(label)
        #results[pm].append(pred)
    #unknown   
    for pm, pred in zip(pmids, predictions):
        results[pm].append(pred)
    
    return results

def load_data(unknown_file_path):
    dataset = load_dataset("csv", data_files = {'unknown': unknown_file_path})
    return dataset
#predict unseen 2022
#dataset = load_data(["2022pubs_cleaned_sep.csv"])
#results , y_pred = predict(test_set)
#results1 = predict([dataset['unknown'][int(i)] for i in range(5000)])
#results2 = predict([dataset['unknown'][int(i)] for i in range(5000,10000)])
#results3 = predict([dataset['unknown'][int(i)] for i in range(10000, 15000)])
#results4 = predict([dataset['unknown'][int(i)] for i in range(15000, 20000)])
#results5 = predict([dataset['unknown'][int(i)] for i in range(20000, 25000)])
#results6 = predict([dataset['unknown'][int(i)] for i in range(25000, 30000)])
results7 = predict([dataset['unknown'][int(i)] for i in range(30000, len(dataset['unknown']))])

In [33]:
results = {}
results.update(results1)
results.update(results2)
results.update(results3)
results.update(results4)
results.update(results5)
results.update(results6)
results.update(results7)
len(results)

33481

In [38]:
#saving 2022 predictions
df = pd.DataFrame.from_dict(results, orient = 'index')
df.to_csv('pubPredictions_22.csv', index = True)

In [28]:
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np

def multi_class_performance(y_true, y_pred):

    mcm = multilabel_confusion_matrix(y_true, y_pred)

    tps = mcm[:, 1, 1]
    tns = mcm[:, 0, 0]

    recall      = tps / (tps + mcm[:, 1, 0])         # Sensitivity
    specificity = tns / (tns + mcm[:, 0, 1])         # Specificity
    precision   = tps / (tps + mcm[:, 0, 1])         # PPV
    return recall, specificity, precision

recall , specificity, precision = multi_class_performance(y_true, y_pred)
print(recall, specificity, precision)

[0.96583144 0.90756303 0.6        0.97142857] [0.92491468 0.96963563 0.98876404 0.99856528] [0.95067265 0.93506494 0.6        0.97142857]


In [ ]:
#validation of joined abstract and title
max_length = 256
ft_model = "PubMedBERT_2021_2epoch/checkpoint-714"
#ft_model = 'final_PubMedBERT/checkpoint-771'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
clf = pipeline("text-classification", model = ft_model, tokenizer = tokenizer)

y_pred_true = collections.defaultdict(list)
y_pred = []
y_true = []
for i in range(len(dataset['validation'])):
    p = clf(dataset['validation']['text'][i], truncation = True, max_length = max_length)[0]['label']
    if p == 'LABEL_0':
        y_pred_true[dataset['validation'][i]['PMID']].append(0)
        y_pred.append(0)
    elif p == 'LABEL_1':
        y_pred_true[dataset['validation'][i]['PMID']].append(1)
        y_pred.append(1)
    elif p == 'LABEL_2':
        print('hi')
        y_pred_true[dataset['validation'][i]['PMID']].append(2)
        y_pred.append(2)
    else:
        print('hi')
        y_pred_true[dataset['validation'][i]['PMID']].append(3)
        y_pred.append(3)
    y_pred_true[dataset['validation'][i]['PMID']].append(dataset['validation'][i]['labels'])
    y_true.append(dataset['validation'][i]['labels'])

In [ ]:
#deployment
y_pred = collections.defaultdict(list)
for i in range(len(dataset['unknown'])):
    p = clf(dataset['unknown']['text'][i], truncation = True, max_length = max_length)[0]
    
    if p['label'] == 'LABEL_0':
        y_pred[dataset['unknown'][i]['PMID']].append(0)
    elif p['label'] == 'LABEL_1':
        y_pred[dataset['unknown'][i]['PMID']].append(1)
    elif p['label'] == 'LABEL_2':
        y_pred[dataset['unknown'][i]['PMID']].append(2)
    else:
        y_pred[dataset['unknown'][i]['PMID']].append(3)
    y_pred[dataset['unknown'][i]['PMID']].append(p['score'])


In [ ]:
prediction = pd.DataFrame.from_dict(y_pred, orient = 'index')
print(prediction)
prediction.to_csv('2022prediction.csv')

In [ ]:
predictions = pd.DataFrame.from_dict(y_pred, orient = 'index', columns = ['Category'])
predictions.index.names = ['PMID']


In [ ]:
predictions.to_csv("2021_RTDesignPreds_longer.csv")

In [ ]:
#prevention setup

def load_data(train_file_path, valid_file_path):
    if train_file_path and valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path, 'validation': valid_file_path})
    elif not valid_file_path:
        dataset = load_dataset("csv", data_files = {'train': train_file_path})
    return dataset
model_checkpoint = 'google/bigbird-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
max_length = 512
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=2,
        #id2label={index: label for index, label in enumerate(labels.names)},
        #label2id={label: index for index, label in enumerate(labels.names)}
    )
    return model
model = model_init()
def tokenize_data(example):
    return tokenizer(example['text'],  padding = 'max_length', truncation=True, max_length=max_length)
dataset = load_data(['trainPrevention.csv'],  ['validPrevention.csv'])
train_tokenized = dataset['train'].map(tokenize_data, batched = True) 
valid_tokenized = dataset['validation'].map(tokenize_data, batched = True)

In [ ]:
#fine-tuning for final classification with whole data
run_name = "PubMedBert_prevention"
def fine_tune(model_checkpoint, output_dir, lr, batch_size, epochs, weight_decay):
    seed = 123
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2)
    training_args = TrainingArguments(
        output_dir = output_dir,
        learning_rate = lr,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs= epochs,
        weight_decay = weight_decay,
        #report_to='wandb',
        #logging_steps = 'epoch',
        #load_best_model_at_end = True,
        #evaluation_strategy = 'epoch',
        save_strategy= 'epoch',
        save_total_limit = 1,
        #run_name= run_name,
        
        )

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    #eval_dataset=valid_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #callbacks = [early_stopping],
     )
    a = trainer.train()
    #wandb.finish()
    return a

fine_tune(model_checkpoint,run_name, 7e-7, 4, 12, 0.001)

In [ ]:
#hyperparameter tuning prevention
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'values': [1]
        },
    'batch_size': {
        'values': [4, 8, 16]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-4,
        'max': 1e-2
    },
    'weight_decay': {
        'values': [0.0, 0.01, 0.001]
    },
}


sweep_config['parameters'] = parameters_dict


In [ ]:
run_name = "BigBird_prevention"

sweep_id = wandb.sweep(sweep_config, project = run_name)

In [ ]:
#preventiohn hyperparam tuning
run_name =  "BigBird_prevention"
def finetune(config=None):
    with wandb.init(config=config):
        config = wandb.config


        training_args = TrainingArguments(
            output_dir=run_name,
            report_to='wandb', 
            num_train_epochs=config.epochs,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=16,
            #save_strategy='epoch',
            evaluation_strategy='epoch',
            logging_strategy='epoch',
            #load_best_model_at_end=True,
            #remove_unused_columns=False,
            #fp16=True,
            #save_total_limit = 1,
            run_name= run_name,
        )
        

        trainer = Trainer(
            model_init=model_init,
            args=training_args,
            train_dataset=train_tokenized,
            eval_dataset=valid_tokenized,
            compute_metrics=compute_metrics_fn
        )


        trainer.train()


In [ ]:
wandb.agent(sweep_id, finetune, count=10)

In [ ]:
#candidate model more epochs prevention
run_name = 'BigBird_prevention'
def train(learning_rate, weight_decay, run_name, epochs, batch_size):
    training_args = TrainingArguments(
        output_dir= run_name,
        report_to='wandb', 
        num_train_epochs= epochs,
        learning_rate= learning_rate,
        weight_decay= weight_decay,
        per_device_train_batch_size= batch_size,
        per_device_eval_batch_size=16,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,
        remove_unused_columns=False,
        #fp16=True,
        save_total_limit = 1,
        run_name= run_name,
    )


    trainer = Trainer(
        model_init= model_init,
        args= training_args,
        train_dataset= train_tokenized,
        eval_dataset= valid_tokenized,
        compute_metrics= compute_metrics_fn
    )


    trainer.train()


#prevention sweep 4
train(7.779e-3, 0.01, run_name, 7, 16)

In [ ]:
#calculate performance metrics experiment
#performance metrics
model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=2
        #id2label={index: label for index, label in enumerate(labels.names)},
        #label2id={label: index for index, label in enumerate(labels.names)}
    )

clf = pipeline("text-classification", model = model_checkpoint, tokenizer = tokenizer)
y_pred = {}
y_true = {}
for i in range(len(dataset['validation'])):
    y_true[i] = dataset['validation']['labels'][i]
    p = clf(dataset['validation']['text'][i], truncation = True, max_length = max_length)[0]['label']
    #print(p)
    if p == 'LABEL_0':
        y_pred[i] = 0
    elif p == 'LABEL_1':
        y_pred[i] = 1
    #elif p == 'LABEL_2':
        #y_pred[i] = 2
    #else:
        #y_pred[i] = 3


In [ ]:
encoding = tokenizer.encode_plus("my name is elaheh", pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

In [ ]:
def confusion_matrix_binary(y_true, y_pred):
    
    tp, fn, tn, fp = 0, 0, 0, 0
   
    for i in range(len(y_true)):
        if y_true[i] == 1 and y_pred[i] == 0:
            fn += 1
        if y_true[i] == 1 and y_pred[i] == 1:
            tp += 1
        if y_true[i] == 0 and y_pred[i] == 0:
            tn += 1
        if y_true[i] == 0 and y_pred[i] == 1:
            fp += 1
    return tp, fn, tn, fp

def metric_binary(tp, fn, tn, fp):
    try:    
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        precision = tp/(tp+fp)
        accuracy = (tp+tn)/(tp+fn+fp+tn)
        f1 = (2*precision*recall)/(precision+recall)

    except ZeroDivisionError:
        pass
    
    print("recall: ",recall, "specificity: ",specificity, "f1: ", f1, "precision:", precision, "accuracy:", accuracy,
          "false negatives: ", fn, "false positive :",fp, "true positive:", tp, "true neg: ", tn)
tp, fn , tn, fp = confusion_matrix_binary(y_true, y_pred)
print(tp, fn , tn, fp)
metric_binary(tp, fn , tn, fp)

In [ ]:
def confusion_matrix(y_true, y_pred):
    
    cm = dict()
    cm = {'1':{'tp':0, 'fp':0, 'tn':0, 'fn':0},
         '2': {'tp':0, 'fp':0, 'tn':0, 'fn':0},
         '3':{'tp':0, 'fp':0, 'tn':0, 'fn':0},
         }
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]: 
            if y_true[i] == 1:
                cm['1']['tp'] += 1
            elif y_true[i] == 0:
                cm['1']['tn'] += 1
                cm['2']['tn'] += 1
                cm['3']['tn'] += 1
            elif y_true[i] == 2:
                cm['2']['tp'] += 1
            else:
                cm['3']['tp'] += 1
        elif y_true[i] != y_pred[i]:
            if y_true[i] == 1:
                cm['1']['fn'] += 1
            elif y_true[i] == 0:
                if y_pred[i] == 1:
                    cm['1']['fp'] += 1
                elif y_pred[i] == 2:
                    cm['2']['fp'] += 1
                elif y_pred[i] == 3:
                    cm['3']['fp'] += 1
            elif y_true[i] == 2:
                cm['2']['fn'] += 1
            else:
                cm['3']['fn'] += 1
            
            
        
        #if y_true[i] == 1 and y_pred[i] == 1:
            #tp += 1
        #if y_true[i] == 0 and y_pred[i] == 0:
            #tn += 1
        #if y_true[i] == 0 and y_pred[i] == 1:
            #fp += 1
    return cm
cm = dict()
cm = confusion_matrix(y_true, y_pred)
for i , val in cm.items():
    recall , specificity, precision, accuracy, f1 = 0,0,0,0,0
    try:    
        recall = val['tp']/(val['tp']+val['fn'])
        specificity = val['tn']/(val['tn']+val['fp'])
        precision = val['tp']/(val['tp']+val['fp'])
        accuracy = (val['tp']+val['tn'])/(val['tp']+val['fn']+val['fp']+val['tn'])
        f1 = (2*precision*recall)/(precision+recall)

    except ZeroDivisionError:
        pass

    print("for class: ", i, "recall: ",recall, "specificity: ",specificity, "f1: ", f1, "precision:", precision, "accuracy:", accuracy,
  "false negatives: ", val['fn'], "false positive :",val['fp'], "true positive:", val['tp'], "true neg: ", val['tn'])

        
#tp, fn, tn, fp = confusion_matrix(y_true, y_pred)

#try:    
    #recall = tp/(tp+fn)
    #specificity = tn/(tn+fp)
    #precision = tp/(tp+fp)
    #accuracy = (tp+tn)/(tp+fn+fp+tn)
    #f1 = (2*precision*recall)/(precision+recall)

#except ZeroDivisionError:
    #pass
    

#print("recall: ",recall, "specificity: ",specificity, "f1: ", f1, "precision:", precision, "accuracy:", accuracy,
      #"false negatives: ", fn, "false positive :",fp, "true positive:", tp, "true neg: ", tn)